In [1]:
import os

# Check if we're in the root directory of rl-llm repo
if not os.path.basename(os.getcwd()) == 'rl-llm':
    # If we're in a subdirectory of rl-llm, find the root and cd to it
    current_path = os.getcwd()
    while os.path.basename(current_path) != 'rl-llm' and os.path.dirname(current_path) != current_path:
        current_path = os.path.dirname(current_path)
    
    if os.path.basename(current_path) == 'rl-llm':
        os.chdir(current_path)
        print(f"Changed directory to: {current_path}")
    else:
        print("Not in rl-llm repository structure")
else:
    print("Already in rl-llm root directory")

print(f"Current directory: {os.getcwd()}")

Changed directory to: /workspace/rl-llm
Current directory: /workspace/rl-llm


In [ ]:
# Import necessary libraries
import os
import torch
import time
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import logging
from src import EnvManager, sample_episodes
from transformers import AutoTokenizer
from trl import AutoModelForCausalLMWithValueHead
from peft import PeftConfig
from typing import List, Dict, Tuple

# Suppress spammy logs from transformers, 
# e.g. "Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation."
logging.getLogger("transformers").setLevel(logging.ERROR)

/workspace/rl-llm/.venv/lib/python3.10/site-packages/gym/envs/registration.py:498: UserWarning: WARN: Overriding environment BabyAI-GoTo-v0 already in registry.
  logger.warn(f"Overriding environment {new_spec.id} already in registry.")
/workspace/rl-llm/.venv/lib/python3.10/site-packages/gym/envs/registration.py:498: UserWarning: WARN: Overriding environment BabyAI-Open-v0 already in registry.
  logger.warn(f"Overriding environment {new_spec.id} already in registry.")
/workspace/rl-llm/.venv/lib/python3.10/site-packages/gym/envs/registration.py:498: UserWarning: WARN: Overriding environment BabyAI-Pickup-v0 already in registry.
  logger.warn(f"Overriding environment {new_spec.id} already in registry.")
/workspace/rl-llm/.venv/lib/python3.10/site-packages/gym/envs/registration.py:498: UserWarning: WARN: Overriding environment BabyAI-PutNext-v0 already in registry.
  logger.warn(f"Overriding environment {new_spec.id} already in registry.")


In [5]:
def load_model_with_peft_and_vhead(model_id, revision=None):
    """
    Load a model with both PEFT adapters and value head
    
    Args:
        model_id: Path to model or HF hub model ID
        device: Device to load the model to
        revision: Specific model revision/commit hash to load
    
    Returns:
        model: The loaded model with adapters and value head
        tokenizer: The associated tokenizer
    """
    
    # Get PEFT config to find base model
    peft_config = PeftConfig.from_pretrained(model_id, revision=revision)
    base_model_id = peft_config.base_model_name_or_path
    
    print(f"Base model: {base_model_id}")
    print(f"Loading with PEFT adapters from: {model_id}")
    if revision:
        print(f"Using revision: {revision}")

    # Load tokenizer
    tokenizer = AutoTokenizer.from_pretrained(base_model_id, padding_side='left')
    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token
    
    # Initialize model with value head from the base model
    model = AutoModelForCausalLMWithValueHead.from_pretrained(
        base_model_id,
        device_map="auto",
    )
    
    # Now load the PEFT adapter weights
    model = model.from_pretrained(model_id, device_map="auto", revision=revision)
    
    return model, tokenizer


def load_base_model_with_vhead(model_id):
    """
    Load a base model with value head
    
    Args:
        model_id: Path to model or HF hub model ID
    
    Returns:
        model: The loaded model with value head
        tokenizer: The associated tokenizer
    """
    # Load tokenizer
    tokenizer = AutoTokenizer.from_pretrained(model_id, padding_side='left')
    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token
    
    print(f"Loading base model with value head from: {model_id}")
    
    # Initialize model with value head
    model = AutoModelForCausalLMWithValueHead.from_pretrained(
        model_id,
        device_map="auto",
    )
    
    return model, tokenizer

def plot_performance_from_results(
    env_ids: List[str],
    evaluation_results: Dict[Tuple[str, str, bool], Dict[str, float]],
    metric: str,  # e.g., "success_rate", "avg_reward", etc.
    context_window: int,
    env_type: str,
    checkpoint_name: str,
):
    """
    Create a bar chart comparing the performance of different model configurations.
    
    Parameters:
      env_ids : List of environment IDs to plot.
      evaluation_results : Dictionary keyed by (model_name, env_id, reasoning_flag) containing performance metrics.
      metric : The key of the performance metric to plot (e.g., "success_rate").
      context_window : Context window size (for display purposes).
      env_type : A string describing the type of environments (e.g. "Seen" or "Unseen").
      checkpoint_name : Name of the checkpoint/model for display.
    """
    # Build dictionaries for each group:
    baseline_non_reasoning_results = {
        env: evaluation_results[("Baseline", env, False)][metric] for env in env_ids
    }
    baseline_reasoning_results = {
        env: evaluation_results[("Baseline", env, True)][metric] for env in env_ids
    }
    # For the trained model, we follow the same convention
    trained_reasoning_results = {
        env: evaluation_results[("Trained", env, True)][metric] for env in env_ids
    }
    trained_non_reasoning_results = {
        env: evaluation_results[("Trained", env, False)][metric] for env in env_ids
    }

    fig, ax = plt.subplots(figsize=(12, 6))
    n_envs = len(env_ids)
    bar_width = 0.2
    index = np.arange(n_envs)

    ax.bar(index - bar_width * 1.5, [baseline_non_reasoning_results[env] for env in env_ids],
           bar_width, label="Baseline (Non-Reasoning)", color='red', alpha=0.7)
    ax.bar(index - bar_width * 0.5, [baseline_reasoning_results[env] for env in env_ids],
           bar_width, label="Baseline (Reasoning)", color='orange', alpha=0.7)
    ax.bar(index + bar_width * 0.5, [trained_reasoning_results[env] for env in env_ids],
           bar_width, label="Reasoning (Trained)", color='blue', alpha=0.7)
    ax.bar(index + bar_width * 1.5, [trained_non_reasoning_results[env] for env in env_ids],
           bar_width, label="Non-Reasoning (Trained)", color='green', alpha=0.7)

    ax.set_xlabel("Environments", fontsize=12)
    ax.set_ylabel(metric.replace('_',' ').title(), fontsize=12)
    ax.set_title(f"Model Performance Comparison\n(Checkpoint: {checkpoint_name}, Context Window: {context_window}, {env_type})",
                 fontsize=14)
    ax.set_xticks(index)
    ax.set_xticklabels(env_ids, rotation=45, ha='right')
    ax.set_ylim(0, 1)
    ax.legend()
    ax.grid(True, linestyle='--', alpha=0.7)

    plt.tight_layout()
    plt.show()
    plt.close()

In [11]:
# Specify models to be evaluated
model_configs = {
    "Baseline": {
        "load_fn": load_base_model_with_vhead,
        "model_id": "meta-llama/Llama-3.2-3B-Instruct", 
    },
    "Trained": {
        "load_fn": load_model_with_peft_and_vhead,
        "model_id": "CatkinChen/final_runs-Reasoning_trial_2_dist_3",   # Replace with the actual Hugging Face ID or local path.
        "revision": None   # Optionally set a revision if needed.
    },
}

In [13]:
# Iterate over each model configuration to load and test with a simple query
for model_name, config in model_configs.items():
    print(f"\nLoading and testing model: {model_name}")

    # Retrieve the loading function, model id, and optionally, the commit hash
    load_fn = config["load_fn"]
    model_id = config["model_id"]
    revision = config.get("revision", None)

    # Load the model and associated tokenizer using the specified function
    if load_fn.__name__ == "load_base_model_with_vhead":
        model, tokenizer = load_fn(model_id)
    else:
        model, tokenizer = load_fn(model_id, revision)

    # Simple query to test the model
    prompt = "You are in a room with a key. The instruction is to: pick up the key"
    
    # Tokenize the input prompt and send tensors to device
    inputs = tokenizer(prompt, return_tensors="pt")
    inputs = {k: v.to(device) for k, v in inputs.items()}

    # Generate output with your text-generation settings
    outputs = model.generate(**inputs, max_new_tokens=50, do_sample=True, temperature=0.7)

    # Decode the generated tokens into text
    output_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    
    # Display the model's output
    print("Model output:")
    print(output_text)


Loading and testing model: Baseline
Loading base model with value head from: meta-llama/Llama-3.2-3B-Instruct


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Model output:
You are in a room with a key. The instruction is to: pick up the key and turn it clockwise to unlock the door.

## Step 1: Observe the key and the door
First, we need to observe the key and the door to understand how they interact. The key seems to be a standard key, and the

Loading and testing model: Trained
Base model: meta-llama/Llama-3.2-3B-Instruct
Loading with PEFT adapters from: CatkinChen/final_runs-Reasoning_trial_2_dist_3


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Model output:
You are in a room with a key. The instruction is to: pick up the key and unlock the door. I can help you with that. I can unlock the door for you.

## Step 1: Understand the instruction
The instruction is to pick up the key and unlock the door.

## Step 2: Identify the role


In [14]:
# Use cuda if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Configuration parameters for text generation
generation_kwargs = {
    "max_new_tokens": 20,
    "do_sample": True,
    "top_k": 50,
    "top_p": 0.9,
    "temperature": 0.7,
    "pad_token_id": tokenizer.pad_token_id,
}

In [15]:
# Define the seen and unseen environments
seen_env_ids = ["BabyAI-GoTo-v0", "BabyAI-Pickup-v0"]
unseen_env_ids = ["BabyAI-Open-v0", "BabyAI-PutNext-v0", "BabyAI-PickUpSeqGoTo-v0"]

# Reasoning flags list
reasoning_flags = [True, False]

# Number of parallel environments to instantiate for each configuration.
num_envs = 6

# Initialise a container for the aggregated results.
evaluation_results = {}

# Function that evaluates a model on a given environment and reasoning flag
def evaluate_model_on_envs(env_id, reasoning_flag, model, tokenizer, generation_kwargs, device, num_episodes=50):
    # Create environment managers for the current env_id and reasoning flag.
    env_managers = [
        EnvManager(
            env_ids=[env_id],
            invalid_action_penalty=-2,
            consecutive_invalid_actions_allowed=5,
            reasoning_flag=reasoning_flag,
            num_dists=3,
        )
        for _ in range(num_envs)
    ]
    
    # Run episodes using sample_episodes
    stats, contexts = sample_episodes(
        envs=env_managers,
        tokenizer=tokenizer,
        model=model,
        generation_kwargs=generation_kwargs,
        device=device,
        number_of_episodes=num_episodes,
        context_window=5,
        reasoning_flag=reasoning_flag,
    )
    
    # Calculate the summary metrics.
    success_rate = sum(stats["success"]) / len(stats["success"])
    avg_reward = sum(stats["rewards"]) / len(stats["rewards"])
    std_reward = torch.std(torch.tensor(stats["rewards"], dtype=torch.float32)).item()
    avg_episode_length = sum(stats["episode_lengths"]) / len(stats["episode_lengths"])
    std_episode_length = torch.std(torch.tensor(stats["episode_lengths"], dtype=torch.float32)).item()
    avg_num_invalid_actions = sum(stats["num_invalid_actions"]) / len(stats["num_invalid_actions"])
    
    # Bundle the results into a dictionary.
    results = {
        "success_rate": success_rate,
        "avg_reward": avg_reward,
        "std_reward": std_reward,
        "avg_episode_length": avg_episode_length,
        "std_episode_length": std_episode_length,
        "avg_num_invalid_actions": avg_num_invalid_actions,
        "contexts": contexts,  # optional: include detailed contexts if needed
    }
    return results

In [17]:
# Main Evaluation Loop
for model_name, config in model_configs.items():
    print(f"Loading model: {model_name}")
    # Load model and tokenizer via the appropriate function.
    if load_fn.__name__ == "load_base_model_with_vhead":
        model, tokenizer = load_fn(model_id)
    else:
        model, tokenizer = load_fn(model_id, revision)
    
    # Update generation kwargs for this model. (Make sure you use its pad_token_id.)
    local_generation_kwargs = {
        "max_new_tokens": 20,
        "do_sample": True,
        "top_k": 50,
        "top_p": 0.9,
        "temperature": 0.7,
        "pad_token_id": tokenizer.pad_token_id,
    }
    
    # Loop over environments (seen and unseen) and reasoning flags.
    # You can separate these groups as needed.
    for env_group_name, env_ids in zip(["Seen", "Unseen"], [seen_env_ids, unseen_env_ids]):
        for env_id in env_ids:
            for reasoning in reasoning_flags:
                print(f"Evaluating {model_name} on {env_group_name} env: {env_id} with reasoning={reasoning}")
                results = evaluate_model_on_envs(
                    env_id=env_id,
                    reasoning_flag=reasoning,
                    model=model,
                    tokenizer=tokenizer,
                    generation_kwargs=local_generation_kwargs,
                    device=device,
                    num_episodes=50  # Adjust as needed.
                )
                # Save results with a composite key.
                evaluation_results[(model_name, env_id, reasoning)] = results

# --- Reporting the results ---
for key, result in evaluation_results.items():
    model_name, env_id, reasoning = key
    print(f"\nResults for {model_name} on {env_id} (reasoning={reasoning}):")
    print(f"  Success Rate: {result['success_rate']}")
    print(f"  Avg Reward: {result['avg_reward']} (Std: {result['std_reward']})")
    print(f"  Avg Episode Length: {result['avg_episode_length']} (Std: {result['std_episode_length']})")
    print(f"  Avg Num. of Invalid Actions: {result['avg_num_invalid_actions']}")

Loading model: Baseline
Base model: meta-llama/Llama-3.2-3B-Instruct
Loading with PEFT adapters from: CatkinChen/final_runs-Reasoning_trial_2_dist_3


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Evaluating Baseline on Seen env: BabyAI-GoTo-v0 with reasoning=True
1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 
Evaluating Baseline on Seen env: BabyAI-GoTo-v0 with reasoning=False
1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 
Evaluating Baseline on Seen env: BabyAI-Pickup-v0 with reasoning=True
1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 
Evaluating Baseline on Seen env: BabyAI-Pickup-v0 with reasoning=False
1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 
Evaluating Baseline on Unseen env: BabyAI-Open-v0 with reasoning=True
1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 

/workspace/rl-llm/.venv/lib/python3.10/site-packages/gym/envs/registration.py:563: UserWarning: WARN: Using the latest versioned environment `BabyAI-PickUpSeqGoTo-v0` instead of the unversioned environment `BabyAI-PickUpSeqGoTo`.
  logger.warn(


1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 
Evaluating Baseline on Unseen env: BabyAI-PickUpSeqGoTo with reasoning=False
1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 
Loading model: Trained
Base model: meta-llama/Llama-3.2-3B-Instruct
Loading with PEFT adapters from: CatkinChen/final_runs-Reasoning_trial_2_dist_3


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Evaluating Trained on Seen env: BabyAI-GoTo-v0 with reasoning=True
1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 
Evaluating Trained on Seen env: BabyAI-GoTo-v0 with reasoning=False
1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 
Evaluating Trained on Seen env: BabyAI-Pickup-v0 with reasoning=True
1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 
Evaluating Trained on Seen env: BabyAI-Pickup-v0 with reasoning=False
1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 
Evaluating Trained on Unseen env: BabyAI-Open-v0 with reasoning=True
1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33

In [ ]:
# Plotting the evaluation results
env_ids_to_plot = seen_env_ids + unseen_env_ids
plot_performance_from_results(
    env_ids=env_ids_to_plot,
    evaluation_results=evaluation_results,
    metric="success_rate",
    context_window=5,          # or any value used in your evaluation
    env_type="All", # "Seen" or "Unseen" or "All"
    checkpoint_name="Final"  # Adjust based on your models
)

In [8]:
env_id = "BabyAI-Pickup-v0"
reasoning_flag = True
env_ids = [env_id]
num_envs = 6
env_managers = [
    EnvManager(
        env_ids, 
        invalid_action_penalty=-2,
        consecutive_invalid_actions_allowed=5,
        reasoning_flag=reasoning_flag,
        num_dists=3,
    )
    for i in range(num_envs)
]

stats, contexts = sample_episodes(
    envs=env_managers,
    tokenizer=tokenizer,
    model=model,
    generation_kwargs=generation_kwargs,
    device=device,
    number_of_episodes=50,
    context_window=5,
    reasoning_flag=reasoning_flag,
)
success_rate = sum(stats["success"]) / len(stats["success"])
avg_reward = sum(stats["rewards"]) / len(stats["rewards"])
std_reward = torch.std(torch.tensor(stats["rewards"], dtype=torch.float32)).item()
avg_episode_length = sum(stats["episode_lengths"]) / len(stats["episode_lengths"])
std_episode_length = torch.std(torch.tensor(stats["episode_lengths"], dtype=torch.float32)).item()
avg_num_invalid_actions = sum(stats["num_invalid_actions"]) / len(stats["num_invalid_actions"])

print(f"Summary: {env_id} with {'Reasoning' if reasoning_flag else 'No Reasoning'}")
print(f"Sample size: {len(stats['success'])}")
print(f"Success Rate: {success_rate:.2f}")
print(f"Average Reward: {avg_reward:.2f} ± {std_reward:.2f}")
print(f"Average Episode Length: {avg_episode_length:.2f} ± {std_episode_length:.2f}")
print(f"Average Number of Invalid Actions: {avg_num_invalid_actions:.2f}")
print()

1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 
Summary: BabyAI-Pickup-v0 with Reasoning
Sample size: 51
Success Rate: 0.88
Average Reward: 0.77 ± 0.29
Average Episode Length: 15.47 ± 18.88
Average Number of Invalid Actions: 0.18



In [10]:
env_id = "BabyAI-PickupTest-v0"
reasoning_flag = True
env_ids = [env_id]
num_envs = 6
env_managers = [
    EnvManager(
        env_ids, 
        invalid_action_penalty=-2,
        consecutive_invalid_actions_allowed=5,
        reasoning_flag=reasoning_flag,
        num_dists=3,
    )
    for i in range(num_envs)
]

stats, contexts = sample_episodes(
    envs=env_managers,
    tokenizer=tokenizer,
    model=model,
    generation_kwargs=generation_kwargs,
    device=device,
    number_of_episodes=50,
    context_window=5,
    reasoning_flag=reasoning_flag,
)
success_rate = sum(stats["success"]) / len(stats["success"])
avg_reward = sum(stats["rewards"]) / len(stats["rewards"])
std_reward = torch.std(torch.tensor(stats["rewards"], dtype=torch.float32)).item()
avg_episode_length = sum(stats["episode_lengths"]) / len(stats["episode_lengths"])
std_episode_length = torch.std(torch.tensor(stats["episode_lengths"], dtype=torch.float32)).item()
avg_num_invalid_actions = sum(stats["num_invalid_actions"]) / len(stats["num_invalid_actions"])

print(f"Summary: {env_id} with {'Reasoning' if reasoning_flag else 'No Reasoning'}")
print(f"Sample size: {len(stats['success'])}")
print(f"Success Rate: {success_rate:.2f}")
print(f"Average Reward: {avg_reward:.2f} ± {std_reward:.2f}")
print(f"Average Episode Length: {avg_episode_length:.2f} ± {std_episode_length:.2f}")
print(f"Average Number of Invalid Actions: {avg_num_invalid_actions:.2f}")
print()

1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 
Summary: BabyAI-PickupTest-v0 with Reasoning
Sample size: 50
Success Rate: 0.94
Average Reward: 0.83 ± 0.22
Average Episode Length: 11.80 ± 14.30
Average Number of Invalid Actions: 0.04



In [11]:
model_id = "CatkinChen/final_runs-No_Reasoning_trial_2_dist_3"
revision = None # "cf75726c11f5b5867107f56efc8f55439635f1c6" 
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model, tokenizer = load_model_with_peft_and_vhead(model_id, revision)

adapter_config.json:   0%|          | 0.00/788 [00:00<?, ?B/s]

Base model: meta-llama/Llama-3.2-3B-Instruct
Loading with PEFT adapters from: CatkinChen/final_runs-No_Reasoning_trial_2_dist_3


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

adapter_model.safetensors:   0%|          | 0.00/36.7M [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/13.8k [00:00<?, ?B/s]

In [11]:
model, tokenizer = load_base_model_with_vhead("meta-llama/Llama-3.2-3B-Instruct")

env_id = "BabyAI-Pickup-v0"
reasoning_flag = True
env_ids = [env_id]
num_envs = 6
env_managers = [
    EnvManager(
        env_ids, 
        invalid_action_penalty=-2,
        consecutive_invalid_actions_allowed=5,
        reasoning_flag=reasoning_flag,
        num_dists=3,
    )
    for i in range(num_envs)
]

stats, contexts = sample_episodes(
    envs=env_managers,
    tokenizer=tokenizer,
    model=model,
    generation_kwargs=generation_kwargs,
    device=device,
    number_of_episodes=50,
    context_window=5,
    reasoning_flag=reasoning_flag,
)
success_rate = sum(stats["success"]) / len(stats["success"])
avg_reward = sum(stats["rewards"]) / len(stats["rewards"])
std_reward = torch.std(torch.tensor(stats["rewards"], dtype=torch.float32)).item()
avg_episode_length = sum(stats["episode_lengths"]) / len(stats["episode_lengths"])
std_episode_length = torch.std(torch.tensor(stats["episode_lengths"], dtype=torch.float32)).item()
avg_num_invalid_actions = sum(stats["num_invalid_actions"]) / len(stats["num_invalid_actions"])

print(f"Summary: {env_id} with {'Reasoning' if reasoning_flag else 'No Reasoning'}")
print(f"Sample size: {len(stats['success'])}")
print(f"Success Rate: {success_rate:.2f}")
print(f"Average Reward: {avg_reward:.2f} ± {std_reward:.2f}")
print(f"Average Episode Length: {avg_episode_length:.2f} ± {std_episode_length:.2f}")
print(f"Average Number of Invalid Actions: {avg_num_invalid_actions:.2f}")
print()

Loading base model with value head from: meta-llama/Llama-3.2-3B-Instruct


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 
Summary: BabyAI-Pickup-v0 with Reasoning
Sample size: 50
Success Rate: 0.16
Average Reward: 0.10 ± 0.25
Average Episode Length: 28.28 ± 26.17
Average Number of Invalid Actions: 4.80



In [12]:
model, tokenizer = load_base_model_with_vhead("meta-llama/Llama-3.2-3B-Instruct")

env_id = "BabyAI-Pickup-v0"
reasoning_flag = False
env_ids = [env_id]
num_envs = 6
env_managers = [
    EnvManager(
        env_ids, 
        invalid_action_penalty=-2,
        consecutive_invalid_actions_allowed=5,
        reasoning_flag=reasoning_flag,
        num_dists=3,
    )
    for i in range(num_envs)
]

stats, contexts = sample_episodes(
    envs=env_managers,
    tokenizer=tokenizer,
    model=model,
    generation_kwargs=generation_kwargs,
    device=device,
    number_of_episodes=50,
    context_window=5,
    reasoning_flag=reasoning_flag,
)
success_rate = sum(stats["success"]) / len(stats["success"])
avg_reward = sum(stats["rewards"]) / len(stats["rewards"])
std_reward = torch.std(torch.tensor(stats["rewards"], dtype=torch.float32)).item()
avg_episode_length = sum(stats["episode_lengths"]) / len(stats["episode_lengths"])
std_episode_length = torch.std(torch.tensor(stats["episode_lengths"], dtype=torch.float32)).item()
avg_num_invalid_actions = sum(stats["num_invalid_actions"]) / len(stats["num_invalid_actions"])

print(f"Summary: {env_id} with {'Reasoning' if reasoning_flag else 'No Reasoning'}")
print(f"Sample size: {len(stats['success'])}")
print(f"Success Rate: {success_rate:.2f}")
print(f"Average Reward: {avg_reward:.2f} ± {std_reward:.2f}")
print(f"Average Episode Length: {avg_episode_length:.2f} ± {std_episode_length:.2f}")
print(f"Average Number of Invalid Actions: {avg_num_invalid_actions:.2f}")
print()

Loading base model with value head from: meta-llama/Llama-3.2-3B-Instruct


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 
Summary: BabyAI-Pickup-v0 with No Reasoning
Sample size: 50
Success Rate: 0.38
Average Reward: 0.24 ± 0.33
Average Episode Length: 52.64 ± 22.58
Average Number of Invalid Actions: 3.82



In [8]:
env_id = "BabyAI-Pickup-v0"
reasoning_flag = False
env_ids = [env_id]
num_envs = 6
env_managers = [
    EnvManager(
        env_ids, 
        invalid_action_penalty=-2,
        consecutive_invalid_actions_allowed=5,
        reasoning_flag=reasoning_flag,
        num_dists=3,
    )
    for i in range(num_envs)
]

stats, contexts = sample_episodes(
    envs=env_managers,
    tokenizer=tokenizer,
    model=model,
    generation_kwargs=generation_kwargs,
    device=device,
    number_of_episodes=50,
    context_window=5,
    reasoning_flag=reasoning_flag,
)
success_rate = sum(stats["success"]) / len(stats["success"])
avg_reward = sum(stats["rewards"]) / len(stats["rewards"])
std_reward = torch.std(torch.tensor(stats["rewards"], dtype=torch.float32)).item()
avg_episode_length = sum(stats["episode_lengths"]) / len(stats["episode_lengths"])
std_episode_length = torch.std(torch.tensor(stats["episode_lengths"], dtype=torch.float32)).item()
avg_num_invalid_actions = sum(stats["num_invalid_actions"]) / len(stats["num_invalid_actions"])

print(f"Summary: {env_id} with {'Reasoning' if reasoning_flag else 'No Reasoning'}")
print(f"Sample size: {len(stats['success'])}")
print(f"Success Rate: {success_rate:.2f}")
print(f"Average Reward: {avg_reward:.2f} ± {std_reward:.2f}")
print(f"Average Episode Length: {avg_episode_length:.2f} ± {std_episode_length:.2f}")
print(f"Average Number of Invalid Actions: {avg_num_invalid_actions:.2f}")
print()

1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 
Summary: BabyAI-Pickup-v0 with No Reasoning
Sample size: 50
Success Rate: 0.86
Average Reward: 0.74 ± 0.33
Average Episode Length: 17.80 ± 21.03
Average Number of Invalid Actions: 0.04



In [9]:
env_id = "BabyAI-PickupTest-v0"
reasoning_flag = False
env_ids = [env_id]
num_envs = 6
env_managers = [
    EnvManager(
        env_ids, 
        invalid_action_penalty=-2,
        consecutive_invalid_actions_allowed=5,
        reasoning_flag=reasoning_flag,
        num_dists=3,
    )
    for i in range(num_envs)
]

stats, contexts = sample_episodes(
    envs=env_managers,
    tokenizer=tokenizer,
    model=model,
    generation_kwargs=generation_kwargs,
    device=device,
    number_of_episodes=50,
    context_window=5,
    reasoning_flag=reasoning_flag,
)
success_rate = sum(stats["success"]) / len(stats["success"])
avg_reward = sum(stats["rewards"]) / len(stats["rewards"])
std_reward = torch.std(torch.tensor(stats["rewards"], dtype=torch.float32)).item()
avg_episode_length = sum(stats["episode_lengths"]) / len(stats["episode_lengths"])
std_episode_length = torch.std(torch.tensor(stats["episode_lengths"], dtype=torch.float32)).item()
avg_num_invalid_actions = sum(stats["num_invalid_actions"]) / len(stats["num_invalid_actions"])

print(f"Summary: {env_id} with {'Reasoning' if reasoning_flag else 'No Reasoning'}")
print(f"Sample size: {len(stats['success'])}")
print(f"Success Rate: {success_rate:.2f}")
print(f"Average Reward: {avg_reward:.2f} ± {std_reward:.2f}")
print(f"Average Episode Length: {avg_episode_length:.2f} ± {std_episode_length:.2f}")
print(f"Average Number of Invalid Actions: {avg_num_invalid_actions:.2f}")
print()

1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 
Summary: BabyAI-PickupTest-v0 with No Reasoning
Sample size: 50
Success Rate: 0.80
Average Reward: 0.70 ± 0.36
Average Episode Length: 19.98 ± 23.05
Average Number of Invalid Actions: 0.04



In [ ]:
# Reasoning Train: Average Reward: 0.81 ± 0.26
# Reasoning Test: Average Reward: 0.76 ± 0.32
# Non-Reasoning Train: Average Reward: 0.74 ± 0.33
# Non-Reasoning Test: Average Reward: 0.70 ± 0.36

In [10]:
# Summary: BabyAI-Pickup-v0 with Reasoning (base model)
# Sample size: 10
# Success Rate: 0.50
# Average Reward: 0.37 ± 0.40
# Average Episode Length: 14.00 ± 9.76
# Average Number of Invalid Actions: 3.60

# Summary: BabyAI-Pickup-v0 with Reasoning (finetuned model but not run until convergence)
# Sample size: 10
# Success Rate: 0.70
# Average Reward: 0.56 ± 0.41
# Average Episode Length: 15.10 ± 10.79
# Average Number of Invalid Actions: 2.50

# Summary: BabyAI-Pickup-v0 with Reasoning (finetuned for longer)
# Sample size: 12
# Success Rate: 0.92
# Average Reward: 0.80 ± 0.26
# Average Episode Length: 9.33 ± 4.29
# Average Number of Invalid Actions: 0.50